In [1]:
from activation import sigmoid, relu, softmax

In [2]:
import numpy as np

In [3]:
softmax(np.array([100,200,300]))

array([1.38389653e-87, 3.72007598e-44, 1.00000000e+00])

In [4]:
softmax(np.array([0.7,70,4]))

array([8.00557267e-31, 1.00000000e+00, 2.17052201e-29])

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import fetch_openml


## 원핫문제에서 // MSE, CEE계산하기 (오차 확인)

### MSE = 1/2*np.sum((y-t)**2)

In [8]:
import pickle
with open('sample_weight.pkl','rb') as f:
    weight = pickle.load(f)

In [9]:
with open('sample_weight.pkl','rb') as f:
    network = pickle.load(f)
    
for i in network.keys():
    print(i)

b2
W1
b1
W2
W3
b3


In [40]:
def predict(x,network):
    input_layer = x
    layers = []
    bias = []
    num_layer = int(len(network.keys())/2)
    for i in range(num_layer):
        layers.append(network['W'+str(i+1)]) #무조건 첫번째꺼
        bias.append(network['b'+str(i+1)]) #무조건 두번째꺼
    # return layers #까지 3바퀴 돌고 >>>>> #어레이를 모았다
    for i in range(num_layer): #한 층씩 들고오기
        if num_layer != i:
            input_layer = sigmoid(np.dot(input_layer,layers[i]) + bias[i]) #relu일때 80%맞을 확률나옴
        else:
            input_layer = np.dot(input_layer,layers[i]) + bias[i]
    # return np.argmax(input_layer,axis=1).astype('str')
    return input_layer

In [19]:
mnist = fetch_openml('mnist_784')
x = mnist['data']
y = mnist['target']

In [20]:
y[predict(x,network) != y] 

24       1
48       9
70       1
80       9
100      5
        ..
69970    5
69975    3
69982    5
69986    3
69998    5
Name: class, Length: 5282, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [21]:
y[24]

'1'

In [22]:
#원핫으로 0,1로 넣었을 때 예시
r = np.array([0,1,0,0,0])

In [23]:
t = predict(x,network)[24]

In [42]:
p = softmax(r)

In [44]:
1/2*np.sum((p-r)**2)

0.22155602429709084

In [ ]:
1/2*np.sum((t-p)**2) 

## 원핫문제에서 // MSE, CEE계산하기 (오차 확인)

### CEE = -np.sum((t*np.log(y))

In [50]:
tf = np.array([0,1,0,0,0,0])
tt = np.array([0,0,0,1,0,0])
yt = softmax(predict(x,network)[0])
yf = softmax(predict(x,network)[24])

In [ ]:
-np.sum((yt*np.log(tt+1e-7))
-np.sum((yf*np.log(tf+1e-7))

## 함수만들기 >> loss함수 만듦

In [59]:
def mse(y,t):
    return np.sum((y-t)**2)/y.size

In [60]:
# CEE(Cross Entropy Error) : (오차)값이 작을수록  정답에 가깝다.
# yi는 신경망의 출력, ti는 정답 레이블(One-Hot인코딩)
def cross_entropy_err(y,t):
    delta = 1e-7
    return -np.sum(np.log(y+delta) * t)/y.shape[0]

In [62]:
y = y.astype(np.int)

In [57]:
y_one = np.zeros_like(y)

In [58]:
y_one

array([0, 0, 0, ..., 0, 0, 0])

In [33]:
y_one = np.zeros((y.size,np.unique(y).size))

In [34]:
for i in range(y.size):
    y_one[y[i]] = 1

In [51]:
mse(predict(x,network),y_one)

0.37606541317106257

In [52]:
cross_entropy_err(predict(x,network),y_one)

200.85241408878449